In [31]:
import mediapipe as mp
import cv2
import csv
import numpy as np
import time

- `drawing_utils` в `mediapipe.solutions` позволяет выводить UA (Unit Actions) на изображение, т.е. рисует точки.
- `holistic` в `mediapipe.solutions` содержит инструменты по извлечению UA на основе изображения.

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

В качестве тестого видео будем использовать съемку с веб-камеры. Если таковая у вас отсутсвует, можно вместо "0" ввести путь к вашему видео.

In [3]:
cap = cv2.VideoCapture(0)

Следующий скрипт запустит запись с веб камеры на 180 фреймов (кадров), затем выключится. Из следующего блока стоит понять, только то, что само изображение записывается в переменную `frame`, а также обратить внимание на строчку `if cv2.waitKey...`. До конца не врубился, что она делает, но она нужна.

In [4]:
num_frames = 0
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow("Video", frame)
    num_frames += 1
    if cv2.waitKey(10) & 0xFF==ord("q"):
        break
    if num_frames == 180:
        break
cv2.destroyAllWindows()

In [15]:
black = (0, 0, 0)
white = (256, 256, 256)
green = (0, 256, 0)
blue = (256, 0, 0)
red = (0, 0, 256)
yellow = (0, 256, 256)
pink = (256, 0, 256)
light_blue = (256, 256, 0)
purple = (256, 0, 128)
orange = (0, 168, 256)

Теперь вновь откроем запись веб-камеры, но на этот раз уже с отображением AU. Воспользуемся двумя методами, которые мы создали в самом начале:

In [6]:
num_frames = 0
with mp_holistic.Holistic() as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(img)

        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=red, thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=green, thickness=1, circle_radius=1)
            )
        cv2.imshow("Video", frame)
        num_frames += 1
        if cv2.waitKey(10) & 0xFF==ord("q"):
            break
        if num_frames == 180:
            break
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Из кода выше мы должны понять, что:
- `mediapipe` в отличии от `opencv` работает с **RGB** изображениями, а не **BGR**.
- координаты AU для всех частей тела записываются в переменную **results** методом `holistic.process(img)`

Вот работа того же самого кода без конвертирования BGR в RGB:

In [7]:
num_frames = 0
with mp_holistic.Holistic() as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        results = holistic.process(frame)

        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=red, thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=green, thickness=1, circle_radius=1)
            )
        cv2.imshow("Video", frame)
        num_frames += 1
        if cv2.waitKey(10) & 0xFF==ord("q"):
            break
        if num_frames == 180:
            break
cv2.destroyAllWindows()

Сам в шоке, но работает вроде нормально, да и быстрее как-будто. В документации было написано конвертировать, но если ему не принципиально, то и нам тоже. **Следует провести тесты на скорость и точность результатов для BGR и RGB форматов.**

**Mediapipe** также имеет *landmakrs* для положения тела, правой и левой руки. Давайте запишем *landmarks* для всего тела, а потом посмотрим как они выглядит отдельно от видеопотока.

In [27]:
land_storage = []
num_frames = 0
with mp_holistic.Holistic() as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        land_storage.append(holistic.process(frame))
        num_frames += 1
        if num_frames == 300:
            break

In [34]:
for f in range(300):
    frame = np.zeros((480, 640, 3))
    results = land_storage[f]
    mp_drawing.draw_landmarks(
        frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
        mp_drawing.DrawingSpec(color=red, thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=green, thickness=1, circle_radius=1)
        )

    mp_drawing.draw_landmarks(
        frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color= purple, thickness=3, circle_radius=1),
        mp_drawing.DrawingSpec(color= green, thickness=3, circle_radius=1)
        )

    mp_drawing.draw_landmarks(
        frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color= orange, thickness=2, circle_radius=2),
        mp_drawing.DrawingSpec(color = green, thickness=2, circle_radius=1)
        )

    mp_drawing.draw_landmarks(
        frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color= light_blue, thickness=2, circle_radius=2),
        mp_drawing.DrawingSpec(color = green, thickness=2, circle_radius=1)
        )
    cv2.imshow("Video", frame)
    time.sleep(0.03)
    if cv2.waitKey(10) and 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

Настала пора посмотреть на данные, которые будет генерить **mediapipe**. Запишем их в отдельный csv файл, чтобы можно было потом обучить простой классификатор какой-нибудь задаче. Создадим csv файл, в который будем записывать AU для правой руки.

In [38]:
num_hand_parameters = 21 # количество точек AU для любой руки в Mediapipe
filename = 'right_hand_data.csv'
landmarks = ["class"]
for val in range(1, num_hand_parameters+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
    
    with open(f'{filename}', mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)  

Мы создали новый csv файл. В нем всего 84 параметра, по 4 на каждую AU. Теперь запишем в него данные. В начале напишите класс, который вы хотите создать в датасете. Затем изображайте перед веб-камерой этот класс правой рукой. Когда запишиться сто кадров с этим жестом, программа перестанет рабоать и вы можете записать какой-нибудь еще жест.

In [42]:
class_name = input("Введите имя класса/лейбла: ")
nums_examples_data = 100
with mp_holistic.Holistic() as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        results = holistic.process(frame)

        row = []
        try:
            right_hand = results.right_hand_landmarks.landmark
            row += list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
        
            row.insert(0, class_name)
            
            with open(f"{filename}", mode = 'a', newline = '') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

            nums_examples_data -= 1
            print(nums_examples_data)
        
        except:
            pass
        
        if nums_examples_data == 0:
            break


99
98
97
96
95
94
93
92
91
90
89
88
87
86
85
84
83
82
81
80
79
78
77
76
75
74
73
72
71
70
69
68
67
66
65
64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


Итак, я записал свой csv файл с двумя классами. Обучим простой классификатор распознавать эти жесты и затем прикрутим эту модель классификатора к распознованию в реальном времени

In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import pickle

from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier
    )


In [47]:
df = pd.read_csv(filename)
X = df.drop("class", axis=1)
y = df["class"]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=124)

pipelines = {
    'RFC':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'GBC':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

fit_models = {}
for algo, pipeline in pipelines.items():
    model_file_name = f'{filename}_{algo}.pkl'
    model = pipeline.fit(x_train, y_train)
    with open(model_file_name, 'wb') as f:
        pickle.dump(model, f)

На выходе у нас появились две обученные модели (рандомого леса и градиентного бустинга). Класс, попробуем теперь детектить в реальном времени наши жесты.

In [50]:
if input("Введите GBC или RFC: ") == "GBC":
    file_model = "right_hand_data.csv_GBC.pkl"
elif input("Введите GBC или RFC: ") == "RFC":
    file_model = "right_hand_data.csv_RFC.pkl"

with open(file_model, 'rb') as f:
    model = pickle.load(f)

count = 200
with mp_holistic.Holistic() as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        results = holistic.process(frame)
        
        try:
            right_hand = results.right_hand_landmarks.landmark
            right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
            X = pd.DataFrame([right_hand_row])
            predict = model.predict(X)[0]

# DISPLAY STATUS BOX
            cv2.rectangle(
                frame, 
                (0,0), 
                (250, 60), 
                (245,117,16), 
                -1)

# DISPLAY CLASS
            cv2.putText(
                frame, 
                'CLASS', 
                (95,12), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                0.5, 
                (0,0,0), 
                1, 
                cv2.LINE_AA)

            cv2.putText(
                frame, 
                predict.split(' ')[0], 
                (90, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA)

            count -= 1
                
        except:
            pass

        cv2.imshow("WebCam", frame)

        if cv2.waitKey(10) and 0xFF == ord('q'):
            break
        if count == 0:
            break

cap.release()
cv2.destroyAllWindows()

/home/liptee/.local/share/virtualenvs/Hidden-Emotion-Recognition-D_wv7OPE/lib64/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/liptee/.local/share/virtualenvs/Hidden-Emotion-Recognition-D_wv7OPE/lib64/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/liptee/.local/share/virtualenvs/Hidden-Emotion-Recognition-D_wv7OPE/lib64/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/liptee/.local/share/virtualenvs/Hidden-Emotion-Recognition-D_wv7OPE/lib64/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/liptee/.local/share/virtua

Не знаю как у вас, а у меня для модели обученной всего на 100 экземплярах обычными классификаторами модель показывает вполне себе внятные результаты. 